# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0         faya  18.3851   42.4509     22.06        35          21   
1        1      farsund  58.0948    6.8047     13.30       100           0   
2        2  new norfolk -42.7826  147.0587     11.72        58          12   
3        3    jamestown  42.0970  -79.2353      5.77        77         100   
4        4      lanzhou  36.0564  103.7922     14.53        48          59   

   Wind Speed Country        Date  
0        2.60      SA  1666108228  
1        7.65      NO  1666108228  
2        1.34      AU  1666108230  
3        9.77      US  1666107934  
4        1.20      CN  1666108230

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
configued_data = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Lat", "Lng"]
)

# Display the map
configued_data

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df.loc[(city_data_df['Max Temp'] > 10) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
filtered_cities = city_data_df.loc[(city_data_df['Max Temp'] > 10) & (city_data_df['Humidity'] < 50)]
clean_city_data = filtered_cities.dropna()

# Display sample data
clean_city_data

City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0          0         faya  18.3851   42.4509     22.06        35          21   
4          4      lanzhou  36.0564  103.7922     14.53        48          59   
22        22   guiratinga -16.3494  -53.7583     38.35        14          22   
29        29  saint-louis  16.3333  -15.0000     38.80        24           4   
36        36      drochia  48.0350   27.8161     13.39        30          73   
..       ...          ...      ...       ...       ...       ...         ...   
542      542        bilma  18.6853   12.9164     31.90        18           0   
545      545       kalabo -14.9700   22.6814     33.55        14         100   
550      550         pali  25.7667   73.3333     27.18        26           0   
554      554     freeport  40.6576  -73.5832     13.43        46          89   
561      561        choma -16.8065   26.9531     26.75        22          94   

     Wind Speed Country        Date  
0          2.60      SA  1666108228  
4          1.20      CN  1666108230  
22         3.10      BR  1666108243  
29         1.78      SN  1666108247  
36         3.46      MD  1666108253  
..          ...     ...         ...  
542        4.17      NE  1666108548  
545        2.90      ZM  1666108550  
550        1.38      IN  1666108552  
554        2.24      US  1666108554  
561        3.63      ZM  1666108557  

[71 rows x 10 columns]

### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_data[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head ()

City Country      Lat       Lng  Humidity Hotel Name
0          faya      SA  18.3851   42.4509        35           
4       lanzhou      CN  36.0564  103.7922        48           
22   guiratinga      BR -16.3494  -53.7583        14           
29  saint-louis      SN  16.3333  -15.0000        24           
36      drochia      MD  48.0350   27.8161        30

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":  "3e0e1a2861fc4b8ea507816659537f4c",  # Add your API key here
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data hotel_df using Jupyter Notebook


Starting hotel search
faya - nearest hotel: No hotel found
lanzhou - nearest hotel: 西园宾馆
guiratinga - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
drochia - nearest hotel: Hotel Confort
nalut - nearest hotel: No hotel found
rafaela - nearest hotel: Parra Hotel & Suites
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
siavonga - nearest hotel: Lake Safari Lodge Annex
nova olimpia - nearest hotel: Olímpia Hotel
palencia - nearest hotel: Hotel Área de Servicio Los Chopos
yidu - nearest hotel: No hotel found
cedar city - nearest hotel: Baymont Inn & Suites
brenham - nearest hotel: Ant Street Inn
almaznyy - nearest hotel: Зверево
sao miguel do araguaia - nearest hotel: Hotel São Miguel
aripuana - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
avera - nearest hotel: No hotel found
ouallam - nearest hotel: No hotel fo

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map_df = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Lat", "Lng"]
)

# Display the map
hotel_map_df

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)